In [ ]:
pip install "numpy>=1.16.5,<1.23.0"

# Import Library

In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.regularizers import l2

# Atur lokasi dataset

In [ ]:
# atur lokasi
dir_dataset = "/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256" # Mendefinisikan path dataset

# total semua data
train_dir = "/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256/Training"
val_dir = "/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256/Validation"

# ================= banyak data train ===========================
train_class = sorted(os.listdir(train_dir))
dict_train = {}

for class_name in train_class:
  class_dir = os.path.join(train_dir, class_name)
  val_count = len(os.listdir(class_dir))
  key = class_name
  value = val_count
  dict_train[key] = value

train_count = sum([len(files) for r, d, files in os.walk(train_dir)])

print (f'''
Banyak data train Early Blight                                  :  {dict_train["Early_Blight"]} data
Banyak data train Late Blight                                   :  {dict_train["Late_Blight"]} data
Banyak data train Healty                                        :  {dict_train["Healthy"]} data
                                          ---------
Total data Train                                                : {train_count} data''')

# ================= Banyak data Val ============================
val_class = sorted(os.listdir(val_dir))
dict_valid = {}

for class_name in val_class:
  class_dir = os.path.join(val_dir, class_name)
  val_count = len(os.listdir(class_dir))
  key = class_name
  value = val_count
  dict_valid[key] = value

val_count = sum([len(files) for r, d, files in os.walk(val_dir)])

print (f'''
Banyak data train Early Blight                                  :  {dict_valid["Early_Blight"]} data
Banyak data train Late Blight                                   :  {dict_valid["Late_Blight"]} data
Banyak data train Healty                                        :  {dict_valid["Healthy"]} data
                                          ---------
Total data Train                                                : {dict_valid} data''')

total_data = train_count + val_count
print (f'''
Total seluruh data                      : {total_data} data''')

# Visualisasi data

In [ ]:
# menggabungkan total data dari validasi dan train seluruh kelas
dict_data = {}
for key in dict_train:
  dict_data[key] = dict_train.get(key, 0) + dict_valid.get(key, 0)

for key in dict_valid:
    if key not in dict_train:
        dict_data[key] = dict_valid[key]

# dict_data

In [ ]:
fig = plt.figure(figsize=(25, 10))
plt.bar(dict_data.keys(), dict_data.values());
plt.title("Jumlah Gambar Tiap Label");
plt.xlabel('Label');
plt.ylabel('Jumlah Gambar');

# Preprocessing

In [ ]:
# split data 20% untuk validasi   
# Mengatur lebar dan tinggi gambar
img_width = 224
img_height = 224
channels = 3
batch_size = 16

Train_datagen = ImageDataGenerator(rescale=1./255)

Validation_datagen = ImageDataGenerator(rescale=1./255)

# Train dan Validation generator  dengan mode categorical
Train_generator = Train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width,img_height),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = True
)

Validation_generator = Validation_datagen.flow_from_directory(
    val_dir,
    target_size = (img_width,img_height),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False
)

# Modeling CNN

In [ ]:
##### Pembuatan model
from keras import regularizers
# inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))

# for layer in inception_model.layers:
#   layer.trainable = False

cnn_model = tf.keras.models.Sequential(
    [    #  inception_model,
     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Flatten(),
     # Droupout
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer=regularizers.l2(0.01)),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(64, activation='relu'),
     tf.keras.layers.Dense(64, activation='relu'),
     tf.keras.layers.Dense(32, activation='relu'),
     tf.keras.layers.Dense(3, activation='softmax')
    ]
)

In [ ]:
# Penggunaan Optimizer 'Adam' dengan learning rate 0.00146
Optimizer = tf.keras.optimizers.Adam(1e-5)
cnn_model.compile(optimizer=Optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy']) 

In [ ]:
# create callbacks
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs['accuracy'] >= 0.9 and logs['val_accuracy'] >= 0.8:
            print("\nReached 80% accuracy and testing accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
callbacks = myCallback()
history = cnn_model.fit(Train_generator,
                    validation_data = Validation_generator,
                    epochs = 60,
                    verbose = 1,
                    callbacks=callbacks)

# Cek precision, f1 score, recall

In [ ]:
# Precission, F1 score, recall
import numpy as np
from sklearn.metrics import recall_score, f1_score, precision_score


val_preds = cnn_model.predict_generator(Validation_generator)
val_preds = np.argmax(val_preds, axis=-1)
val_true = Validation_generator.classes

# ==================== Precission
precision = precision_score(val_true, val_preds, average='weighted')
precision = round(float(precision), 2)

# ==================== Recall
recall = tf.keras.metrics.Recall()(val_true, val_preds)
recall = tf.reduce_mean(recall)
recall = round(float(recall), 2)

# ==================== F1 socre
f1 = f1_score(val_true, val_preds, average='weighted')
f1 = round(f1,2)

print(f'''
Hasil dari Precission, recall, dan F1 score sebagai berikut
--------------------------
Recall     : {recall} / {recall*100}%
--------------------------
Precission : {precision} / {precision*100}%
--------------------------
F1 score   : {f1} / {f1*100}%
--------------------------''')
# precision.dtype

# grafik akurasi train dan val, loss train dan val

In [ ]:
# Membuat plot akurasi model VGG16
plt.figure(figsize=(10,4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('CNN model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()

print()

# Membuat plot loss model VGG16
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()

# Test Model

In [ ]:
Train_generator.class_indices

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
# mengambil data acak 15
img_dir = "/kaggle/input/potato-disease-leaf-datasetpld/PLD_3_Classes_256/Testing/Early_Blight"
img_list = os.listdir(img_dir)
random_size = 25
img_sources = random.sample(img_list, random_size)

i = 1
total_high = []
for img_source in img_sources:
    img_path = os.path.join(img_dir, img_source)
    image_load = image.load_img(img_path, target_size=(img_width, img_height))
#     imgplot = plt.imshow(image_load)
    x = image.img_to_array(image_load)
    x = np.expand_dims(x, axis=0)
    probs = cnn_model.predict(x)


# img_source = "/kaggle/input/rice-leafs-disease-dataset/RiceLeafsDisease/validation/healthy/healthy_val (12).jpg"
# image_load = image.load_img(img_source, target_size = (img_width,img_height))
# imgplot = plt.imshow(image_load)
# x = image.img_to_array(image_load)
# x = np.expand_dims(x, axis=0)
# probs = cnn_model.predict(x)
    print (f'''iterasi ke - {i} : ''')
    # convert percent 
    probs = np.clip(probs,0,1)
    percent = probs * 100

    # print softmax probabilities
    high = np.argmax(probs)
    
    
    if high == 0:
        print ("Early Blight")
        print(f'''Akurasi : {np.round(percent[0,0],2)}''')
        total_high.append(high)
    elif high == 1:
        print("Healty")
        print(f'''Akurasi : {np.round(percent[0,1],2)}''')
        total_high.append(high)
    else:
        print("Late Blight")
        print(f'''Akurasi : {np.round(percent[0,2],2)}''')
        total_high.append(high)
    print("\n--------------------------------------------")
        
    i += 1

# print (total_high)

In [ ]:
# initial
dir_name = os.path.basename(img_dir)
dir_name_trainVal = os.path.basename(os.path.dirname(img_dir))
# hitung list
count_data0 = total_high.count(0)
count_data1 = total_high.count(1)
count_data2 = total_high.count(2)
count_data3 = total_high.count(3)
count_data4 = total_high.count(4)
count_data5 = total_high.count(5)

list_count = [count_data0,count_data1,count_data2,count_data3,count_data4,count_data5]
max_list_count = max(list_count)

# kondisi
if dir_name == 'Early_Blight':
    idx_name = count_data0
elif dir_name == 'Healthy':
    idx_name = count_data1
else:
    idx_name = count_data2
    

print(f'''\n*********************************************************************
Early_Blight           : {count_data0} data
Healthy                : {count_data1} data
Late_Blight            : {count_data2} data

Total data test        : {idx_name}/{random_size} data {dir_name} {dir_name_trainVal}

''')